In [1]:
!nvidia-smi

Sun Mar 16 13:57:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#!pip install accelerate
!pip install transformers
!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy3
!pip install peft
#!pip install flash_attn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [4]:
from datasets import load_dataset

dataset = load_dataset('ai-forever/MERA', 'rcb', split='validation')
dataset

Dataset({
    features: ['instruction', 'inputs', 'outputs', 'meta'],
    num_rows: 220
})

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

model = AutoModelForCausalLM.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct', attn_implementation='sdpa', device_map='cuda')
model.eval()

tokenizer = AutoTokenizer.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct')
gen_config = GenerationConfig.from_pretrained('RefalMachine/RuadaptQwen2.5-1.5B-instruct')
gen_config

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/2.28M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

GenerationConfig {
  "do_sample": true,
  "eos_token_id": 145111,
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}

In [6]:
gen_config.max_new_tokens = 1
gen_config.do_sample = False

In [8]:
def sample2messages(sample):
    return [{'role': 'user', 'content': sample['instruction'].format(**sample['inputs'])}]

dataset[0], sample2messages(dataset[0])

({'instruction': 'Приведено описание ситуации и гипотеза. Ситуация: "{premise}" Гипотеза: "{hypothesis}". Определи отношение гипотезы к ситуации, выбери один из трех вариантов: 1 - гипотеза следует из ситуации, 2 - гипотеза противоречит ситуации, 3 - гипотеза независима от ситуации. В ответ напиши только цифру 1, 2 или 3, больше ничего не добавляй.',
  'inputs': {'premise': 'Его подозревают в причинении смерти по неосторожности и незаконном хранении и ношении огнестрельного оружия.',
   'hypothesis': 'Он по неосторожности убил кого-то.'},
  'outputs': '3',
  'meta': {'verb': 'подозревать',
   'negation': 'no_negation',
   'genre': 'kp',
   'id': 0}},
 [{'role': 'user',
   'content': 'Приведено описание ситуации и гипотеза. Ситуация: "Его подозревают в причинении смерти по неосторожности и незаконном хранении и ношении огнестрельного оружия." Гипотеза: "Он по неосторожности убил кого-то.". Определи отношение гипотезы к ситуации, выбери один из трех вариантов: 1 - гипотеза следует из сит

In [14]:
tokenizer.apply_chat_template(sample2messages(dataset[0]), add_generation_prompt=True, return_tensors='pt')

tensor([[145110,    872,    198,  53645,  26991,  13103,  72181, 105277, 102002,
           7587, 122024,     13, 125899,     25,    330,  55091,  65249, 108370,
          99179,   5805, 100946,  83098, 100710,  17686,  18658, 114684,  67642,
           7587, 109462,  93862, 137998,   7587,  99167, 134689, 140182, 103400,
           1189, 133397,  13039,  31885,   1478,     25,    330, 106426,  17686,
          18658, 114684,  67642, 110736, 101251,     12,  24634,  36911, 137404,
         102700, 122025,   7665, 102002,     11,  62129,  99409,  99401,  23064,
         101123, 109579,     25,    220,     16,    481, 122024, 100497,  23064,
         102002,     11,    220,     17,    481, 122024, 119733, 102002,     11,
            220,     18,    481, 122024, 106070,   1478,  20264, 102002,     13,
          22933,  92647,  90005, 127721,  73626, 103062,  19763,    220,     16,
             11,    220,     17,  45077,    220,     18,     11,  99565, 100051,
          18658, 121664,  12

In [18]:
import torch
inputs = tokenizer.apply_chat_template(sample2messages(dataset[0]), add_generation_prompt=True, return_tensors='pt')
inputs = inputs.to(model.device)
with torch.no_grad():
    res = model.generate(inputs, generation_config=gen_config).detach().cpu().numpy()[0]
res, tokenizer.decode(res[inputs.shape[1]:])

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


(array([145110,    872,    198,  53645,  26991,  13103,  72181, 105277,
        102002,   7587, 122024,     13, 125899,     25,    330,  55091,
         65249, 108370,  99179,   5805, 100946,  83098, 100710,  17686,
         18658, 114684,  67642,   7587, 109462,  93862, 137998,   7587,
         99167, 134689, 140182, 103400,   1189, 133397,  13039,  31885,
          1478,     25,    330, 106426,  17686,  18658, 114684,  67642,
        110736, 101251,     12,  24634,  36911, 137404, 102700, 122025,
          7665, 102002,     11,  62129,  99409,  99401,  23064, 101123,
        109579,     25,    220,     16,    481, 122024, 100497,  23064,
        102002,     11,    220,     17,    481, 122024, 119733, 102002,
            11,    220,     18,    481, 122024, 106070,   1478,  20264,
        102002,     13,  22933,  92647,  90005, 127721,  73626, 103062,
         19763,    220,     16,     11,    220,     17,  45077,    220,
            18,     11,  99565, 100051,  18658, 121664,  12141, 

In [20]:
import torch
def generate(sample):
    inputs = tokenizer.apply_chat_template(sample2messages(sample), add_generation_prompt=True, return_tensors='pt') #shape тут 1 x длина последовательности
    inputs = inputs.to(model.device)
    with torch.no_grad():
        res = model.generate(inputs, generation_config=gen_config).detach().cpu().numpy()[0]

    return tokenizer.decode(res[inputs.shape[1]:]) #только новые токены

In [22]:
import numpy as np
from tqdm import tqdm

metrics = []
for sample in tqdm(dataset):
    predict = generate(sample)
    metrics.append(predict == str(sample['outputs']))

print('Accuracy = ', np.mean(metrics))

  0%|          | 0/220 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 220/220 [00:29<00:00,  7.34it/s]

Accuracy =  0.4
